In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1,2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1,2


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from ganime.data.base import load_dataset
from ganime.model.p2p.p2p_v2 import P2P
import tensorflow as tf
from tqdm import tqdm
tf.get_logger().setLevel('ERROR')

In [4]:
train_ds, test_ds, input_shape = load_dataset("moving_mnist", "../../data", batch_size=16)

2022-04-26 02:23:07.252188: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-26 02:23:07.895321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14252 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:25:00.0, compute capability: 8.6
2022-04-26 02:23:07.896220: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14252 MB memory:  -> device: 1, name: NVIDIA RTX A4000, pci bus id: 0000:41:00.0, compute capability: 8.6


In [5]:
train_ds_iterator = train_ds.as_numpy_iterator()
x = next(train_ds_iterator)[0]

In [6]:
model = P2P()

In [7]:
from ganime.visualization.videos import display_videos

In [9]:
n_epochs = 10

for epoch in tqdm(range(n_epochs)):
    epoch_mse = 0
    epoch_kld = 0
    epoch_align = 0
    epoch_cpc = 0

    for x in train_ds.as_numpy_iterator():
        x = x[0]

        mse, kld, cpc, align = model(x)
        epoch_mse += mse
        epoch_kld += kld
        epoch_cpc += cpc
        epoch_align += align
    print("EPOCH", epoch)
    print("epoch mse", epoch_mse)
    print("epoch kld", epoch_kld)
    print("epoch cpc", epoch_cpc)
    print("epoch align", epoch_align)

 10%|█████████████████▉                                                                                                                                                                 | 1/10 [01:35<14:18, 95.37s/it]

EPOCH 0
epoch mse tf.Tensor(23.948185, shape=(), dtype=float32)
epoch kld tf.Tensor(0.0, shape=(), dtype=float32)
epoch cpc tf.Tensor(0.0, shape=(), dtype=float32)
epoch align tf.Tensor(19812.799, shape=(), dtype=float32)


 20%|███████████████████████████████████▊                                                                                                                                               | 2/10 [03:10<12:43, 95.38s/it]

EPOCH 1
epoch mse tf.Tensor(23.9482, shape=(), dtype=float32)
epoch kld tf.Tensor(0.0, shape=(), dtype=float32)
epoch cpc tf.Tensor(0.0, shape=(), dtype=float32)
epoch align tf.Tensor(19810.451, shape=(), dtype=float32)


 30%|█████████████████████████████████████████████████████▋                                                                                                                             | 3/10 [04:46<11:07, 95.39s/it]

...

EPOCH 7
epoch mse tf.Tensor(23.948185, shape=(), dtype=float32)
epoch kld tf.Tensor(0.0, shape=(), dtype=float32)
epoch cpc tf.Tensor(0.0, shape=(), dtype=float32)
epoch align tf.Tensor(19775.38, shape=(), dtype=float32)


 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 9/10 [14:18<01:35, 95.43s/it]

EPOCH 8
epoch mse tf.Tensor(23.948162, shape=(), dtype=float32)
epoch kld tf.Tensor(0.0, shape=(), dtype=float32)
epoch cpc tf.Tensor(0.0, shape=(), dtype=float32)
epoch align tf.Tensor(19769.643, shape=(), dtype=float32)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [15:54<00:00, 95.45s/it]

EPOCH 9
epoch mse tf.Tensor(23.9482, shape=(), dtype=float32)
epoch kld tf.Tensor(0.0, shape=(), dtype=float32)
epoch cpc tf.Tensor(0.0, shape=(), dtype=float32)
epoch align tf.Tensor(19679.299, shape=(), dtype=float32)
